In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
import os
DIVERSITY_UTILITY = float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = 0.5

###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

0.5

In [5]:
len(df)

100000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_minority','T_majority','Y']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_minority,T_majority,Y
99974,0,12,0.02,10,12,0
99914,0,16,0.03,10,16,0
99913,0,16,0.03,12,16,0
99650,0,17,0.06,14,17,0
99653,0,17,0.06,13,17,0
...,...,...,...,...,...,...
99940,1,94,1.21,94,107,0
99949,1,101,1.23,101,118,1
98552,1,98,1.25,98,114,1
99977,1,112,1.25,112,129,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','Y','ml_outcomes']].groupby(['R','T','Y']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','Y']].groupby(['R','T','Y']).count().reset_index()
df_count.columns = ['R','T','Y','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,Y,ml_outcomes,N
0,342,0,0,12,0,0.02,1
3,341,3,0,16,0,0.06,2
5,340,5,0,17,1,0.06,1
2,339,2,0,15,0,0.07,1
13,338,13,0,21,1,0.07,1
...,...,...,...,...,...,...,...
91,4,91,0,60,1,666.00,1110
97,3,97,0,63,1,666.90,1026
95,2,95,0,62,1,672.00,1050
99,1,99,0,64,1,674.69,1007


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,Y,ml_outcomes,N
0,342,0,0,12,0,0.02,1
1,333,1,0,13,0,0.10,1
2,339,2,0,15,0,0.07,1
3,341,3,0,16,0,0.06,2
4,326,4,0,17,0,0.42,7
...,...,...,...,...,...,...,...
338,298,338,1,95,1,1.19,1
339,292,339,1,97,1,1.25,1
340,293,340,1,98,1,1.25,1
341,295,341,1,101,1,1.23,1


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['Y'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in Y stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['Y'])]
    key2 = (row['R'],row['T'], 1-row['Y'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(41, 302)

In [21]:
majority_pass_boards = []
majority_fail_boards = []
minority_pass_boards = []
minority_fail_boards = []

for key in vars_cache:
    r, t, Y = key
    if Y == 1 and r==0:
        majority_pass_boards.append(key)
    elif Y == 0 and r==0:
        majority_fail_boards.append(key)
    elif Y == 1 and r==1:
        minority_pass_boards.append(key)
    elif Y == 0 and r==1:
        minority_fail_boards.append(key)

len(majority_pass_boards),len(majority_fail_boards),len(minority_pass_boards),len(minority_fail_boards)

(100, 87, 80, 76)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','Y']].groupby(['R','Y']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['Y'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['Y'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

344

In [24]:
#Of those who pass the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in majority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in majority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,1.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
338,338,1.0
339,339,1.0
340,340,1.0
341,341,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
93,0,93,0,61,1,677.04,1092,93,1.0
99,1,99,0,64,1,674.69,1007,99,1.0
95,2,95,0,62,1,672.00,1050,95,1.0
97,3,97,0,63,1,666.90,1026,97,1.0
91,4,91,0,60,1,666.00,1110,91,1.0
...,...,...,...,...,...,...,...,...,...
13,338,13,0,21,1,0.07,1,13,0.0
2,339,2,0,15,0,0.07,1,2,0.0
5,340,5,0,17,1,0.06,1,5,0.0
3,341,3,0,16,0,0.06,2,3,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
187,331,187,1,7,0,0.25,1,187,0.0
188,330,188,1,12,0,0.31,1,188,0.0
0,342,0,0,12,0,0.02,1,0,0.0
189,324,189,1,13,0,0.60,2,189,0.0
1,333,1,0,13,0,0.10,1,1,1.0
...,...,...,...,...,...,...,...,...,...
182,281,182,0,113,1,1.98,2,182,1.0
183,309,183,0,115,1,1.00,1,183,1.0
184,308,184,0,116,1,1.00,1,184,1.0
185,311,185,0,118,1,0.98,1,185,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
93,0,93,0,61,1,677.04,1092,93,1.0
99,1,99,0,64,1,674.69,1007,99,1.0
95,2,95,0,62,1,672.00,1050,95,1.0
97,3,97,0,63,1,666.90,1026,97,1.0
91,4,91,0,60,1,666.00,1110,91,1.0
...,...,...,...,...,...,...,...,...,...
13,338,13,0,21,1,0.07,1,13,0.0
2,339,2,0,15,0,0.07,1,2,0.0
5,340,5,0,17,1,0.06,1,5,0.0
3,341,3,0,16,0,0.06,2,3,0.0


In [35]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['Y'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['Y'].astype(str)

In [36]:
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
93,0,93,0,61,1,677.04,1092,93,1.0,0_61_1
99,1,99,0,64,1,674.69,1007,99,1.0,0_64_1
95,2,95,0,62,1,672.00,1050,95,1.0,0_62_1
97,3,97,0,63,1,666.90,1026,97,1.0,0_63_1
91,4,91,0,60,1,666.00,1110,91,1.0,0_60_1
...,...,...,...,...,...,...,...,...,...,...
13,338,13,0,21,1,0.07,1,13,0.0,0_21_1
2,339,2,0,15,0,0.07,1,2,0.0,0_15_0
5,340,5,0,17,1,0.06,1,5,0.0,0_17_1
3,341,3,0,16,0,0.06,2,3,0.0,0_16_0


In [37]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_minority_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()

In [38]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()

In [39]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
0,342,0,0,12,0,0.02,1,0,0.0,0_12_0
44,134,44,0,37,0,125.80,629,44,0.0,0_37_0
20,261,20,0,25,0,5.22,58,20,0.0,0_25_0
59,125,59,0,44,1,152.40,508,59,0.0,0_44_1
46,121,46,0,38,0,164.43,783,46,0.0,0_38_0
...,...,...,...,...,...,...,...,...,...,...
141,133,141,0,85,1,126.49,139,141,1.0,0_85_1
116,139,116,0,73,0,115.83,143,116,1.0,0_73_0
143,143,143,0,86,1,109.48,119,143,1.0,0_86_1
137,119,137,0,83,1,173.70,193,137,1.0,0_83_1


In [40]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
210,172,210,1,27,0,62.90,185,210,0.0,1_27_0
204,208,204,1,24,0,24.00,75,204,0.0,1_24_0
223,202,223,1,33,1,29.26,77,223,0.0,1_33_1
206,196,206,1,25,0,34.32,104,206,0.0,1_25_0
225,194,225,1,34,1,37.24,98,225,0.0,1_34_1
...,...,...,...,...,...,...,...,...,...,...
294,179,294,1,69,0,53.35,55,294,1.0,1_69_0
307,178,307,1,75,1,56.18,53,307,1.0,1_75_1
309,177,309,1,76,1,56.71,53,309,1.0,1_76_1
270,83,270,1,57,0,256.04,346,270,1.0,1_57_0


In [41]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.49999999999999983

In [42]:
admit_decisions[admit_decisions['Y']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.680089
1,0.680089
